In [37]:
import pandas as pd
import sys

# sys.path.append("../")

from lib.uid import gen_uid
from lib.clean import clean_names, clean_races, clean_sexes

In [38]:
def read_personnel_and_employment_hist():
    dfa = pd.read_csv("../../../data/FL/2023-4-13/employment-20231220144255.csv", encoding="latin1")

    dfb = pd.read_csv("../../../data/FL/2023-4-13/person_data-20231220144255.csv", encoding="latin1")

    df = pd.merge(dfa, dfb, on="person_nbr")

    dfc = pd.read_csv("../../../data/FL/2023-4-13/agency_map.csv", encoding="latin1")

    df = pd.merge(df, dfc, on="agcy_nbr", how="left")
    return df


In [39]:
df = read_personnel_and_employment_hist()

/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_83314/1656383686.py:2: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  dfa = pd.read_csv("../../../data/FL/2023-4-13/employment-20231220144255.csv", encoding="latin1")


In [40]:
df = df.rename(columns={"sex_code": "sex", "race_code": "race", "employ_start_date": "hire_date", "separation_date": "left_date", "agcy_name": "agency"})

df = df.pipe(clean_names, ["first_name", "last_name", "middle_name"]).pipe(gen_uid, ["first_name", "last_name", "middle_name", "agency"])

def clean_dates(df):
    # Convert the 'ST_DATE' column to datetime
    df['hire_date'] = pd.to_datetime(df['hire_date'])
    
    # Format the dates as 'Month Day, Year'
    df['hire_date'] = df['hire_date'].dt.strftime('%m/%d/%Y')

    df['left_date'] = pd.to_datetime(df['left_date'])


    df['left_date'] = df['left_date'].dt.strftime('%m/%d/%Y')
    return df

df = df.pipe(clean_dates).pipe(clean_races, ["race"]).pipe(clean_sexes, ["sex"])

df["state"] = "FL"

standardize_from_lookup_table: unmatched sequences:
  {'oth', 'na'}
standardize_from_lookup_table: unmatched sequences:
  {'', 'u'}


In [41]:
df

,employment_nbr,person_nbr,agcy_nbr,employ_type,employ_class,hire_date,left_date,separation_code,employ_tea,fingerprint_card_process_date,...,last_name,first_name,middle_name,sex,race,birth_year,Unnamed: 0,agency,uid,state
0,1,97866,378,PT,LE,01/18/2000,09/05/2001,ANM,N,4/20/2000 0:00:00,...,piwowarski,michael,p,male,white,1969.0,526,Oak Hill Police Department,b4cbffe2a853f9474ec550c2ccbaff90,FL
1,291513,97866,378,PT,LE,09/05/2001,04/13/2002,ANM,N,4/20/2000 0:00:00,...,piwowarski,michael,p,male,white,1969.0,526,Oak Hill Police Department,b4cbffe2a853f9474ec550c2ccbaff90,FL
2,176523,97866,162,FT,LE,10/15/1992,07/09/1997,Mis,N,10/15/1992 0:00:00,...,piwowarski,michael,p,male,white,1969.0,348,Orange County Sheriff's Office,dd83af4c6302340ce7ccfc9ac46a0d9f,FL
3,176524,97866,162,PT,LE,02/18/1992,10/15/1992,TWA,N,2/18/1992 0:00:00,...,piwowarski,michael,p,male,white,1969.0,348,Orange County Sheriff's Office,dd83af4c6302340ce7ccfc9ac46a0d9f,FL
4,261156,97866,393,PT,LE,09/04/1998,09/06/1999,FP,N,NaN,...,piwowarski,michael,p,male,white,1969.0,831,Pahokee Police Department,417a46bf049b6a59cc7ed0809336f7bb,FL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629841,643901,542440,62,FT,Corr,03/24/2023,NaN,NaN,Y,2/27/2023 0:00:00,...,wilson,zy rieana,demetra kantrell,female,black,2003.0,324,Department Of Corrections,d99377169fc413574b6e6c8339466b13,FL
629842,643907,542460,178,FT,LE,04/03/2023,NaN,NaN,Y,3/29/2023 0:00:00,...,arellano,jason,,male,hispanic,1985.0,502,Volusia Sheriff's Office,1a2d0f6d2f64093b6414ec2b9cfde210,FL
629843,643910,416647,290,FT,LE,04/03/2023,NaN,NaN,N,1/9/2023 0:00:00,...,ibarra,prudenciano,,male,hispanic,1977.0,514,Holly Hill Police Department,047f0b790beb67838b8e991781c37bc4,FL
629844,643911,542766,128,FT,LE,03/13/2023,NaN,NaN,Y,NaN,...,hope,kiandre,m,male,black,1996.0,331,Dixie County Sheriff's Office,eea413fb8ecbf330765c311a42803932,FL


In [42]:
df_events = df[["uid", "agency", "state", "hire_date", "left_date"]]


df_events = df_events[~((df_events.left_date.fillna("") == ""))]

# Unpivot the DataFrame
df_long = df_events.melt(id_vars=['uid', 'agency', 'state'], 
                  value_vars=['hire_date', 'left_date'], 
                  var_name='event_type', 
                  value_name='event_date')

# Generate event_uid using gen_uid function
df_long = df_long.pipe(gen_uid, ['uid', 'event_type', 'event_date'], "event_uid")

events_df= df_long[df_long.duplicated(subset=["uid"], keep=False)]

# officers that we know have moved
events_df.sort_values("uid")

events_df.to_csv("./../../../data/FL/match/events_florida_dropped_na_left_dates.csv", index=False)

In [ ]:
dup_uids_list = [x for x in events_df["uid"]]

per_df = df[["first_name", "middle_name", "last_name", "sex", "race", "birth_year", "uid", "state"]]

per_df = per_df[per_df.uid.isin(dup_uids_list)]

per_df.to_csv("./../../../data/FL/match/personnel_florida_dropped_na_left_dates.csv", index=False)